In [0]:
%load_ext autoreload
%autoreload 2

import matplotlib
from matplotlib import pyplot
pyplot.ion()
pyplot.style.use('dark_background')

import torch as th
from rl.algs.dqn import *

In [2]:
from rl.algs import experiment
from rl.algs import model
from rl.algs.envs import grid_world

BATCH_SIZE = 32
TRAIN_START_STEP = 3 * BATCH_SIZE
NUM_TRAIN_STEPS = 10000 # 17
REPLAY_SIZE = 100000
TARGET_NETWORK_UPDATE_FREQ = 500


env = environment.GridWorldWrapped(grid_shape=(4, 4, 1))
conv_specs = {
  model.ConvSpec(4, 1, 32),
}
fc_specs = (64,)
model_params = {
  'conv_specs': conv_specs,
  'fc_specs': fc_specs,
}

EPS_DECAY = util.Schedule((0, TRAIN_START_STEP, NUM_TRAIN_STEPS / 1, NUM_TRAIN_STEPS),
                          (1, 1, 0.0, 0.0))
EPS_RANDOM = util.Schedule((0,), (1.0,))
adam_params = {
  'fn': th.optim.Adam,
  'kwargs': {
    'lr': 0.005,
  }
}
rmsprop_params = {
  'fn': th.optim.RMSprop,
  'kwargs': {
    'lr': 0.00025,
    'momentum': 0.95,
    'alpha': 0.95,
    'eps': 0.01,
  }
}

policy_params = {
    'opt_params': rmsprop_params,
    'eps_sched': EPS_DECAY,  # TODO: change
    'target_update_freq': TARGET_NETWORK_UPDATE_FREQ,
}
exp = experiment.Experiment(env, model.QNetwork, model_params,
                            Dqn, policy_params)

print(exp.policy.model)

rb = ReplayBuffer(REPLAY_SIZE)
for step in range(TRAIN_START_STEP):
  sars = env.step(exp.policy.get_action)
  rb.add(sars)

eps_rew = 0
for step in range(TRAIN_START_STEP, NUM_TRAIN_STEPS):
  sars = env.step(exp.policy.get_action)
  rb.add(sars)
    
  experience_batch = rb.sample(BATCH_SIZE)
  m = exp.policy.update(experience_batch)
  
  eps_rew += sars.r
  if sars.s1 is None:
    m['r_per_eps'] = eps_rew
    eps_rew = 0
  if step % 100 == 0 or sars.s1 is None:
    for k, v in m.items():
      exp.plt.add_data(k, step, v)
      
# Expect about r_per_eps=-50 in pure random agent.

(1, 1, 32)
QNetwork(
  (convs): Sequential(
    (0): Conv2d(1, 32, kernel_size=(4, 4), stride=(1, 1))
    (1): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=32, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=4, bias=True)
  )
)


In [3]:
opt = grid_world.GridWorld((4, 4)).optimal_expected_reward()
print(f'optimal expected r: {opt}')

optimal expected r: 1.6666666666666665


In [4]:
env.show_model_policy(exp.policy.model)

[[ 0.          0.23253331 -0.9832648  -2.10604334]
 [ 0.09648687 -0.91693467 -1.81020594 -2.88292694]
 [-0.84658396 -1.75224888 -2.76791334 -3.8289485 ]
 [-1.65294409 -2.72266769 -3.71129036 -4.72605419]]
[['⟲' '←' '←' '←']
 ['↑' '↑' '←' '←']
 ['↑' '←' '↑' '←']
 ['↑' '↑' '←' '←']]
[[-0.19834287  0.          0.12362084 -1.0377903 ]
 [-0.99650681  0.01657796 -0.96666861 -2.04466391]
 [-1.96815658 -1.06661427 -2.05238676 -2.94815159]
 [-2.68560505 -2.15101266 -2.75188971 -3.57549214]]
[['→' '⟲' '←' '←']
 ['↑' '↑' '←' '←']
 ['↑' '↑' '←' '←']
 ['→' '↑' '←' '←']]
[[-1.03045809 -0.04109663  0.          0.04311869]
 [-2.00755167 -0.93429738  0.01845449 -0.93474472]
 [-2.91365027 -1.98717356 -1.0551095  -1.94019783]
 [-3.50515938 -2.86753869 -2.02168608 -2.75784183]]
[['→' '→' '⟲' '←']
 ['↑' '↑' '↑' '←']
 ['→' '↑' '↑' '←']
 ['↑' '↑' '↑' '←']]
[[-1.98953307 -1.08157504 -0.08470879  0.        ]
 [-2.97966433 -1.98144341 -0.95511532  0.01743859]
 [-3.87784815 -2.88562655 -1.98370981 -1.03387463]
 

In [5]:
# Debug the action-value function for a given state.
s = [[0, 0, 0, 0],
     [0, 0, 0, 0],
     [0, 0, 1, 0],
     [0, 0, 2, 0]]
s = np.expand_dims(np.expand_dims(s, 0), -1)
print(exp.policy.model(util.to_variable(s)))
print(exp.policy.target_model(util.to_variable(s)))

tensor([[-1.9617, -2.2265,  0.0421, -2.1250]],
       device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([[-2.2406, -2.0398, -0.0952, -1.9214]],
       device='cuda:0', grad_fn=<ThAddmmBackward>)


In [6]:
# Examine the replay buffer.
terminals = rb.filter_by(sars_filter(r=0))
print(len(terminals))

# action-value at 's' was wrong. Search the replay buffer for occurrences of
# the correct transition. It's likely missing, which led the NN to overfit
# on missing data.
trans_s = rb.filter_by(sars_filter(s=s))
trans_s_a = rb.filter_by(sars_filter(s=s,a=2))
print(f'Num with (s): {len(trans_s)}. Num with (s, a): {len(trans_s_a)}')


1704
Num with (s): 62. Num with (s, a): 43


In [7]:
env.visualize(exp.policy, steps=20)

array([[0, 0, 0, 0],
       [0, 0, 2, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 0]])
array([[0, 0, 0, 0],
       [0, 0, 2, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 2],
       [1, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 2],
       [0, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 2],
       [0, 0, 1, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 2],
       [0, 0, 0, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 0],
       [1, 0, 0, 2],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 0],
       [0, 1, 0, 2],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 0],
       [0, 0, 1, 2],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 0],
  